In [1]:
import sys 
sys.path.append("/scratch/m/murray/dtolgay/")
from tools import constants

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt

In [2]:
# def read_interpolated_Lline(base_dir, galaxy_name, galaxy_type, redshift, directory_name, file_name):
     
#     lines = [
#         "L_ly_alpha",  # [erg s^-1]
#         "L_h_alpha", # [erg s^-1]
#         "L_h_beta", # [erg s^-1]
#         "L_co_10", # [K km s^-1 pc^2] 
#         "L_co_21", # [K km s^-1 pc^2] 
#         "L_co_32", # [K km s^-1 pc^2] 
#         "L_co_43", # [K km s^-1 pc^2] 
#         "L_co_54", # [K km s^-1 pc^2] 
#         "L_co_65", # [K km s^-1 pc^2] 
#         "L_co_76", # [K km s^-1 pc^2] 
#         "L_co_87", # [K km s^-1 pc^2] 
#         "L_13co",  # [K km s^-1 pc^2] 
#         "L_c2", # [erg s^-1]
#         "L_o3_88", # [erg s^-1]
#         "L_o3_5006", # [erg s^-1]
#         "L_o3_4958", # [erg s^-1] 
#     ]

#     gas_column_names = [
#         "x",                                    # pc 
#         "y",                                    # pc 
#         "z",                                    # pc 
#         "smoothing_length",                     # pc 
#         "mass",                                 # Msolar
#         "metallicity",                          # Zsolar
#         "temperature",                          # Kelvin
#         "vx",                                   # km/s
#         "vy",                                   # km/s
#         "vz",                                   # km/s
#         "hden",                                 # 1/cm3
#         "radius",                               # pc 
#         "sfr",                                  # Msolar / year
#         "turbulence",                           # km/s
#         "density",                              # gr/cm3
#         "mu_theoretical",                       # 1
#         "average_sobolev_smoothingLength",      # pc 
#         "index",                                # 1
#         "isrf",                                 # G0
#     ] + lines


#     fdir = f'{base_dir}/{galaxy_type}/z{redshift}/{galaxy_name}/{directory_name}/{file_name}'

#     gas = pd.DataFrame(
#         np.loadtxt(fname=fdir), 
#         columns=gas_column_names
#     )

#     return gas


def read_otherProperties(
        galaxy_name, 
        galaxy_type, 
        redshift, 
        directory_name, 
        file_name,
        base_fdir = f"/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius"
    ):
    
    column_names = [
        'x',                # (pc)
        'y',                # (pc)
        'z',                # (pc)
        'smoothing_length', # (pc)
        'mass',             # (Msolar)
        'metallicity',      # (Zsolar)
        'temperature',      # (K)
        'vx',               # (km/s)
        'vy',               # (km/s)
        'vz',               # (km/s)
        'hden',             # (cm^-3)
        'radius',           # (pc)
        'sfr',              # (Msolar/yr)
        'turbulence',       # (km/s)
        'density',          # (gr/cm^-3)
        'mu_theoretical',   # (1)
        'average_sobolev_smoothingLength',  # (pc)
        'index',            # [1]
        'isrf',             # [G0]
        'fh2',              # [1] 
        'fCO',              # [1]
    ]

    ####
    data = pd.DataFrame(
        np.loadtxt(f'{base_fdir}/{galaxy_type}/z{redshift}/{galaxy_name}/{directory_name}/{file_name}'),
        columns = column_names    
    )

    volume = data['mass'] / (data['density'] * constants.gr2Msolar) # cm^3
    h_mass = data['hden'] * volume * constants.proton_mass * constants.kg2Msolar # Msolar
    
    data['h2_mass'] = data['fh2'] * h_mass    # Msolar
    data['co_mass'] = data['fCO'] * h_mass    # Msolar
    
    
    return data

In [4]:
%%time
base_dir = "/home/m/murray/dtolgay/scratch/post_processing_fire_outputs/skirt/runs_hden_radius"

directory_name = "voronoi_1e6"
redshift = "0.0"

galaxy_name = "gal22"
galaxy_type = "firebox"
# galaxy_name = "m12i_res7100_md"
# galaxy_type = "zoom_in"

# nearestND = read_interpolated_Lline(
#     base_dir,
#     galaxy_name, 
#     galaxy_type, 
#     redshift, 
#     directory_name, 
#     file_name = "L_line_averageSobolevH_nearestNDInterpolator_flux2Luminosity.txt"
# )

nearestND = read_otherProperties(
    galaxy_name, 
    galaxy_type, 
    redshift, 
    directory_name, 
    file_name = "otherProperties_averageSobolevH_nearestNDInterpolator.txt",
    base_fdir = base_dir    
)
print("nearestND read")

# hybridND = read_interpolated_Lline(
#     base_dir,
#     galaxy_name, 
#     galaxy_type, 
#     redshift, 
#     directory_name, 
#     file_name = "L_line_averageSobolevH_hybridInterpolator_flux2Luminosity.txt"
# )

hybridND = read_otherProperties(
    galaxy_name, 
    galaxy_type, 
    redshift, 
    directory_name, 
    file_name = "otherProperties_averageSobolevH_hybridInterpolator.txt",
    base_fdir = base_dir    
)
print("hybridND read")


nearestND read
hybridND read
CPU times: user 4.54 s, sys: 242 ms, total: 4.78 s
Wall time: 4.82 s


In [5]:
hybridND.keys()

Index(['x', 'y', 'z', 'smoothing_length', 'mass', 'metallicity', 'temperature',
       'vx', 'vy', 'vz', 'hden', 'radius', 'sfr', 'turbulence', 'density',
       'mu_theoretical', 'average_sobolev_smoothingLength', 'index', 'isrf',
       'fh2', 'fCO', 'h2_mass', 'co_mass'],
      dtype='object')

In [6]:
column_name = "h2_mass"

sum_nearestND = sum(nearestND[column_name])
sum_hybrid = sum(hybridND[column_name])

print(f"sfr: {sum(nearestND['sfr'])}")
print(f"nearest: {sum_nearestND/1e8} --- hybrid: {sum_hybrid/1e8}")

sfr: 2.203638999999999
nearest: 5.469934052925942 --- hybrid: 3.8902745322129966
